In [ ]:
import geopandas as gpd
from geopandas import GeoDataFrame
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import libpysal
from tobler.util import h3fy
from h3 import h3
import descartes


import os
from os.path import isfile, join
from pathlib import Path
from os import listdir
import os
os.getcwd()

path =  Path(os.getcwd())
root = path.parent.absolute()


h3_level = 10

In [ ]:
gdf = gpd.read_file(root / 'X.data' / 'LA_County_City_Boundaries'/ 'LA_County_City_Boundaries.shp')
gdf.sample(5)

mask = gdf.CITY_LABEL.isin(['Long Beach'])

gdf_lb = gdf[mask]
gdf_lb.sample(2)

In [ ]:
gdf_hex = h3fy(gdf_lb, resolution = h3_level)
gdf_hex.shape

In [ ]:
import contextily as ctx
ax = gdf_hex.plot(markersize=0.01, alpha = .3, edgecolors= "black", figsize=(8, 10))

ctx.add_basemap(ax)
plt.show()

In [ ]:
#gdf_hex[['hex_id', 'geometry']].to_file((root / 'X.data' / 'h3_data' / 'base_map'/ 'base_map.shp'))

In [ ]:
crash_dir = root / 'X.data' / 'raw_crashes'

onlyfiles = [f for f in listdir(crash_dir) if isfile(join(crash_dir, f))]

appended_data = []

for file in onlyfiles:
    print(file)
    temp = pd.read_csv(crash_dir / file, low_memory = False)
    appended_data.append(temp)

collision_df = pd.concat(appended_data)
collision_df.columns
#collision_df.CITY.unique()

In [ ]:
#collision_df = collision_df.mask(('ACCIDENT_YEAR' == 2018) and ('CITY' == 'Long Beach'))
coll_lb_18 = collision_df.query('ACCIDENT_YEAR == 2018 and CITY == "LONG BEACH" and DAY_OF_WEEK == 6')
coll_lb_18.sample()

In [ ]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(coll_lb_18.POINT_X, coll_lb_18.POINT_Y)]
gdf_lb = GeoDataFrame(coll_lb_18, geometry=geometry)

gdf_lb.set_crs(epsg=4326, inplace=True).sample(2)

In [ ]:
base = gdf_hex.plot(markersize=0.01, alpha = .3, edgecolors= "black", figsize=(18, 20))
gdf_lb.to_crs(epsg=3857).plot(ax=base, marker='o', color='red', markersize=5)

ctx.add_basemap(base)
plt.show()

In [ ]:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row.POINT_Y, row.POINT_X, h3_level)

pd_dict = {}

coll_lb_18['hex_id'] = coll_lb_18.apply(lat_lng_to_h3, axis=1)

hex_la_18 = coll_lb_18.groupby(['hex_id']).hex_id.agg('count').to_frame('count').reset_index()
hex_la_18.sample(3)

In [ ]:
gdf_hex = gdf_hex.merge(hex_la_18, on = 'hex_id', how = 'left')
gdf_hex.sample(2)

In [ ]:
gdf_hex['collision_count'] = gdf_hex['count'].fillna(0)
gdf_hex.sort_values('collision_count')

In [ ]:
base = gdf_hex.plot('collision_count',alpha = .2, edgecolors= "black", figsize=(15, 18))
gdf_lb.to_crs(epsg=3857).plot(ax=base, marker='o', color='red', markersize=5)

ctx.add_basemap(base)
plt.show()

In [ ]:
def rking_neighbors(row, skins = 1):
    neighbors = h3.k_ring(row.hex_id, skins)
    neighbors_list = list(neighbors)
    return(neighbors_list)

gdf_hex['hex_neighbors_1_ids'] = gdf_hex.apply(rking_neighbors, axis=1)
gdf_hex.sample(2)

In [ ]:
def neihbor_average(row):
    #print(row.hex_neighbors_1_ids)
    neighbor_list = row.hex_neighbors_1_ids
    neighbor_len = len(neighbor_list)
    neighbor_df = gdf_hex[gdf_hex['hex_id'].isin(neighbor_list)]
    smooth_var = sum(neighbor_df['collision_count'])/neighbor_len
    return(smooth_var)

In [ ]:
gdf_hex['neighbor_average_collision_count'] = gdf_hex.apply(neihbor_average, axis=1)
gdf_hex.sample(2)

In [ ]:
base = gdf_hex.plot('neighbor_average_collision_count',alpha = .3, edgecolors= "black", figsize=(18, 20))
gdf_lb.to_crs(epsg=3857).plot(ax=base, marker='o', color='red', markersize=5)

ctx.add_basemap(base)
plt.show()

In [ ]:
import osmnx as ox
#ox.config(use_cache=True, log_console=True)
#G = ox.graph_from_place('Long Beach, CA, USA', network_type='drive')
#Gc = ox.consolidate_intersections(ox.project_graph(G))
#nodes = ox.graph_to_gdfs(Gc, edges=False)
#nodes[['x', 'y', 'lat', 'lon']]

In [ ]:
display(nodes.sample(2))
nodes.shape

In [ ]:
nodes.highway.value_counts()

In [ ]:
geometry = [Point(xy) for xy in zip(nodes.lon, nodes.lat)]
gdf_nodes = GeoDataFrame(nodes, geometry=geometry)

display(gdf_nodes.set_crs(epsg=4326, inplace=True).sample(2))
gdf_nodes.to_crs(epsg=3857).sample(2)

In [ ]:
ox.config(use_cache=True, log_console=True)
#G = ox.graph_from_place('Long Beach, CA, USA', network_type='drive')
#Gc = ox.consolidate_intersections(ox.project_graph(G))
#edges = ox.graph_to_gdfs(Gc, nodes=False)

In [ ]:
edges.sample(2)

In [ ]:
gdf_hex_nodes_inner = gpd.sjoin(gdf_hex, gdf_nodes.to_crs(epsg=3857), how = 'inner')
gdf_hex_nodes_inner.shape

In [ ]:
gdf_hex_edges_inner = gpd.sjoin(gdf_hex, edges.to_crs(epsg=3857), how = 'inner')
gdf_hex_edges_inner.reset_index(inplace = True)

gdf_hex_edges_inner = gdf_hex_edges_inner[['hex_id', 'geometry']].drop_duplicates()
gdf_hex_edges_inner.shape

In [ ]:
gdf_hex_nodes_inner.sample()

In [ ]:
base = gdf_hex_edges_inner.plot(alpha = .3, edgecolors= "black", figsize=(15, 18))
edges.to_crs(epsg=3857).plot(ax=base, color='red', markersize=2)

ctx.add_basemap(base)
plt.show()

In [ ]:
edges.oneway.value_counts()

In [ ]:
base = gdf_hex.plot(alpha = .3, edgecolors= "black", figsize=(15, 18))
edges.to_crs(epsg=3857).plot('oneway', ax=base, markersize=5)

ctx.add_basemap(base)
plt.show()


In [ ]:
gdf_hex_hwy = gpd.sjoin(gdf_hex, edges[['geometry', 'highway']].to_crs(epsg=3857), how='inner', predicate='intersects')
gdf_hex_hwy.sample(4)

In [ ]:
#create hierarchy for roads
residential                    10677
tertiary                        2337
secondary                       1837
primary                          928
unclassified                     322
motorway_link                    293
motorway                         153
trunk                             36
primary_link                      29
[residential, unclassified]       21
trunk_link                        20
secondary_link                    17
tertiary_link                     15
[residential, tertiary]           12
[tertiary, unclassified]           2
[secondary, motorway_link]         1
[trunk_link, motorway_link]        1
[trunk, unclassified]              1
[trunk, motorway]                  1


In [ ]:
sorter = ['motorway_link'
                      ,'motorway'
                      ,'primary'
                      ,'secondary'
                      ,'residential'
                      ,'primary_link'
                      ,'secondary_link'
                      ,'tertiary'
                      ,'trunk'
                      ,'unclassified'
                      ,'other']

gdf_hex_hwy['highway_updated'] = 'other'
gdf_hex_hwy['highway_updated'][gdf_hex_hwy['highway'].isin(sorter)] = gdf_hex_hwy['highway']
gdf_hex_hwy.highway_updated.value_counts()
sorterIndex = dict(zip(sorter, range(len(sorter))))

gdf_hex_hwy['highway_rank'] = gdf_hex_hwy.highway_updated.map(sorterIndex)
#gdf_hex_hwy.sort_values(by=['hex_id', 'highway_rank']).head()
gdf_hex_hwy = gdf_hex_hwy.groupby('hex_id').first()
gdf_hex_hwy.reset_index(inplace = True)
gdf_hex_hwy.sample(3)

In [ ]:
gdf_hex = gdf_hex.merge(gdf_hex_hwy[['hex_id', 'highway']], on ='hex_id', how = 'left')

In [ ]:
gdf_hex.sample()